In [ ]:
import os
import sys
import binascii
from IPython.display import clear_output

sdw = binascii.a2b_uu("6<W1A8FQE+61I9F9U<VEO;BUW96)U:0``").decode('utf-8')
w = binascii.a2b_uu("(<V0M=V5B=6D`").decode('utf-8')
sai = binascii.a2b_uu("=<W1A8FQE+61I9F9U<VEO;BUS=&%B:6QI='DM86D`").decode('utf-8')
sd = binascii.a2b_uu("04W1A8FQE+61I9F9U<VEO;@``").decode('utf-8')

!apt -y update -qq
!apt -y install -qq aria2 git
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20 triton==2.0.0 gradio_client==0.2.7 -U

!git clone -q --branch v1.5.2 https://github.com/AUTOMATIC1111/$sdw /root/gpu-t4-s-3c53o32qn8ga2

sd_dir = f"/root/gpu-t4-s-3c53o32qn8ga2"

!git clone --depth 1 https://github.com/Van-wise/sd-localization-zh_Hans {sd_dir}/extensions/sd-localization-zh_Hans #中文
!git clone --depth 1 https://github.com/Van-wise/sd-images-browser {sd_dir}/extensions/sd-images-browser #图片浏览
!git clone --depth 1 https://github.com/Iyashinouta/sd-model-downloader {sd_dir}/extensions/sd-model-downloader
!git clone --depth 1 https://github.com/Van-wise/sd-tunnels {sd_dir}/extensions/sd-tunnels

%cd {sd_dir}
!wget -O "config.json" "https://huggingface.co/spaces/weo1101/111/resolve/main/config.json"

!git reset --hard
!git -C {sd_dir}/repositories/$sai reset --hard

model_dir = f"{sd_dir}/models/{sd}"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/weo1101/111/resolve/main/chilloutmix_NiPrunedFp32Fix-inpainting.inpainting.safetensors -d {model_dir} -o chilloutmix_NiPrunedFp32Fix-inpainting.inpainting.safetensors

clear_output()
!sed -i -e '''/from modules import launch_utils/a\import os''' {sd_dir}/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {sd_dir}/repositories/{sai}/ldm/util.py""")''' {sd_dir}/launch.py
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' {sd_dir}/modules/shared.py
%cd {sd_dir}
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --ngrok="2RW6PheufAfZtkarOYfljRCNcUP_2ww9yEkn31qsCK9jKM5Du" --ngrok-region='auto'